In [1]:
import pandas as pd
import plotly.express as px

In [2]:
data = pd.read_csv(r'C:/Users/carol/Downloads/ages_dataset.csv')
data.head(3)

,Id,Name,Short description,Gender,Country,Occupation,Birth year,Death year,Manner of death,Age of death,Associated Countries,Associated Country Coordinates (Lat/Lon),Associated Country Life Expectancy
0,Q23,George Washington,1st president of the United States (1732–1799),Male,United States of America; Kingdom of Great Bri...,Politician,1732,1799.0,natural causes,67.0,"['United Kingdom', 'United States']","[(55.378051, -3.435973), (37.09024, -95.712891)]","[81.3, 78.5]"
1,Q42,Douglas Adams,English writer and humorist,Male,United Kingdom,Artist,1952,2001.0,natural causes,49.0,['United Kingdom'],"[(55.378051, -3.435973)]",[81.3]
2,Q91,Abraham Lincoln,16th president of the United States (1809-1865),Male,United States of America,Politician,1809,1865.0,homicide,56.0,['United States'],"[(37.09024, -95.712891)]",[78.5]


In [3]:
data.shape

(1223009, 13)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1223009 entries, 0 to 1223008
Data columns (total 13 columns):
 #   Column                                    Non-Null Count    Dtype  
---  ------                                    --------------    -----  
 0   Id                                        1223009 non-null  object 
 1   Name                                      1223009 non-null  object 
 2   Short description                         1155109 non-null  object 
 3   Gender                                    1089363 non-null  object 
 4   Country                                   887500 non-null   object 
 5   Occupation                                1016095 non-null  object 
 6   Birth year                                1223009 non-null  int64  
 7   Death year                                1223008 non-null  float64
 8   Manner of death                           53603 non-null    object 
 9   Age of death                              1223008 non-null  float64
 10  Associ

#### A partir das informações acima, já podemos observar que em muitas colunas, os dados não foram totalmente preenchidos, veremos então como trabalhar com cada coluna da melhor forma possível

In [5]:
#verificando dados vazios
data.isnull().sum()

Id                                                0
Name                                              0
Short description                             67900
Gender                                       133646
Country                                      335509
Occupation                                   206914
Birth year                                        0
Death year                                        1
Manner of death                             1169406
Age of death                                      1
Associated Countries                         368693
Associated Country Coordinates (Lat/Lon)     368693
Associated Country Life Expectancy           368930
dtype: int64

In [6]:
# verificando do que a coluna fala
data['Short description']

0             1st president of the United States (1732–1799)
1                                English writer and humorist
2            16th president of the United States (1809-1865)
3                  Austrian composer of the Classical period
4                     German classical and romantic composer
                                 ...                        
1223004                              Frans model (1907-1996)
1223005          xugador de baloncestu canadianu (1948–2004)
1223006    German navy officer and world war II U-boat co...
1223007    English artist, working in stained glass, prin...
1223008                                       German painter
Name: Short description, Length: 1223009, dtype: object

In [7]:
data.Gender.value_counts()

Male                                              981365
Female                                            107830
Transgender Female                                    79
Transgender Male                                      36
Eunuch; Male                                          18
Intersex                                               7
Female; Male                                           7
Eunuch                                                 3
Transgender Male; Female                               3
Intersex; Female                                       2
Non-Binary                                             2
Intersex; Male                                         2
Transgender Female; Male                               2
Female; Female                                         1
Transgender Female; Intersex                           1
Transgender Female; Female                             1
Transgender Male; Male                                 1
Intersex; Transgender Male     

In [13]:
paises = data.Country.value_counts().nlargest(20)
paises

United States of America                                       152761
Germany                                                         95081
France                                                          78666
United Kingdom; United Kingdom of Great Britain and Ireland     29684
Sweden                                                          26915
Spain                                                           26084
United Kingdom                                                  22498
Canada                                                          22386
Poland                                                          19323
United Kingdom of Great Britain and Ireland                     19026
Italy; Kingdom of Italy                                         18781
Kingdom of the Netherlands                                      18710
Japan                                                           18490
Norway                                                          16762
Austria             

In [17]:
fig = px.bar(x=paises.index, y=paises.values,labels={'x': 'Países', 'y': 'Número de Pessoas por País'},
             title='20 Países com Mais Dados')

fig.update_xaxes(tickangle=45)

fig.show()

In [18]:
ocupacao = data.Occupation.value_counts().nlargest(20)
ocupacao

Artist                281512
Politician            195390
Athlete               110943
Researcher             90709
Military personnel     52911
Religious figure       37086
Businessperson         19529
Architect              17865
Journalist             16002
Teacher                15693
Physician              13253
Engineer               13160
Judge                   8454
Lawyer                  7184
Jurist                  4601
Aristocrat              3670
Entrepreneur            2926
Philosopher             2819
Translator              2742
Publisher               2475
Name: Occupation, dtype: int64

In [20]:
fig = px.bar(x=ocupacao.index, y=ocupacao.values,labels={'x': 'Ocupação', 'y': 'Número de Pessoas por Ocupação'},
             title='20 Ocupações Mais Comuns')

fig.update_xaxes(tickangle=45)

fig.show()

In [23]:
morte = data['Manner of death'].value_counts().nlargest(8)
morte

natural causes                         33532
suicide                                 5814
accident                                5044
homicide                                4692
capital punishment                      3415
unnatural death                          308
death in battle                          250
capital punishment; unnatural death       79
Name: Manner of death, dtype: int64

In [25]:
fig = px.bar(x=morte.index, y=morte.values,labels={'x': 'Tipos de Morte', 'y': 'Número de Pessoas por Tipo de Morte'},
             title='Os Tipos de Morte Mais Comuns')

fig.update_xaxes(tickangle=45)

fig.show()